<a href="https://colab.research.google.com/github/Naveed101633/Agentic-AI-Concepts/blob/main/Handsoff_Vs_Orchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Here You will get complete concept of Handsoff and Orchestrator Agents Example* *and* I hope you will easily get the logic of both... By (Naveed Ahmed)

# HandOff Detail Example

# Import OpenAI-Agents-SDk

In [ ]:
!pip install -qU openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00


# Async Func

In [ ]:
# ✅ Imports
import asyncio
import nest_asyncio

nest_asyncio.apply()

# Global Level Agent Setting

In [ ]:
# ✅ Imports
import asyncio
import nest_asyncio
from google.colab import userdata
from openai import AsyncOpenAI
from agents import (
    Agent,
    Runner,
    function_tool,
    set_default_openai_api,
    set_default_openai_client,
    set_tracing_disabled
)

# ✅ Fix event loop issue in Colab
nest_asyncio.apply()

# ✅ Gemini model setup
BASE_URL = 'https://generativelanguage.googleapis.com/v1beta/openai/'
API_KEY = userdata.get('GOOGLE_API_KEY')
MODEL_NAME = "gemini-2.0-flash"

client = AsyncOpenAI(base_url=BASE_URL, api_key=API_KEY)
set_default_openai_client(client=client, use_for_tracing=False)
set_default_openai_api("chat_completions")
set_tracing_disabled(disabled=True)

# Creating Tools or Function for Calling

***Creating Tools/Function***

In [ ]:
# Function for Product Search
@function_tool
def list_womens_clothing(category: str = "all") -> str:
    inventory = {
        "dresses": ["Red Evening Dress - $120", "Floral Summer Dress - $80"],
        "tops": ["Silk Blouse - $60", "White Crop Top - $35"],
        "all": ["Red Evening Dress - $120", "Floral Summer Dress - $80",
                "Silk Blouse - $60", "White Crop Top - $35"]
    }
    items = inventory.get(category.lower(), inventory["all"])
    return "\n".join(items)



# Function for Support Agent

@function_tool
def answer_product_query(question: str) -> str:
    q = question.lower()
    if "color" in q:
        return "This item is available in red, blue, and black."
    elif "size" in q:
        return "We have sizes XS, S, M, L, and XL."
    elif any(keyword in q for keyword in ["discount", "negotiate", "deal", "offer"]):
        return "We offer a 10% discount on orders above $100."
    else:
        return "Please clarify your question about the product."



# Function of Payment Handling
@function_tool
def process_billing(name: str = "Unknown", email: str = "unknown@example.com", payment_method: str = "credit card") -> str:
    return f"Billing processed for {name} using {payment_method}. A confirmation has been sent to {email}."

# Building Shopping, Support, Billing Agents through Handoffs

***Let's Creating Agents Now***

In [ ]:
# ✅ Agents
shopping_agent = Agent(
    name="Shopping Assistant",
    instructions="""
You help users explore women’s clothing. Use the clothing tool to list products for the requested category.
Format the response exactly as: '🛍️ Shopping Agent: OK. I found two dresses: Red Evening Dress for $120 and Floral Summer Dress for $80.' when the category is 'dresses'.
For other categories, adapt the count and items accordingly, keeping the same structure.
    """,
    tools=[list_womens_clothing],
    model=MODEL_NAME
)

support_agent = Agent(
    name="Support Agent",
    instructions="""
You handle questions about item details or pricing. Use your tool to answer.
For discount-related questions, format the response exactly as: '📦 Support Agent: We offer a 10% discount on orders above $100.'
For other questions, adapt the response using the tool’s output with the prefix '📦 Support Output: '.
    """,
    tools=[answer_product_query],
    model=MODEL_NAME
)

billing_agent = Agent(
    name="Billing Agent",
    instructions="""
You process billing using your tool. Use default values for name and email if not provided, and accept 'credit card' as the payment method.
Format the response exactly as: '💳 Billing Agent: OK. I've processed the billing using the default name and email with the provided credit card. A confirmation has been sent to the default email.'
    """,
    tools=[process_billing],
    model=MODEL_NAME
)

# Triage Agent based on Handsoff

In [ ]:
async def main():
    triage_agent = Agent(
        name="Triage Agent",
        instructions="""
You route user queries to one of the following agents based on the query content:
- If the user mentions clothing, dresses, tops, or wants to browse/shop, hand off to the Shopping Assistant.
- If the user asks about size, color, discounts, deals, or price, hand off to the Support Agent.
- If the user mentions checkout, payment, or billing, hand off to the Billing Agent.
If unsure, ask the user to clarify their request.
        """,
        handoffs=[shopping_agent, support_agent, billing_agent],
        model=MODEL_NAME
    )

    # Test 1: Shopping
    result1 = await Runner.run(triage_agent, "Can you show me some summer dresses?")
    print(result1.final_output)

    # Test 2: Support
    result2 = await Runner.run(triage_agent, "Is there a discount if I buy two items?")
    print( result2.final_output)

    # Test 3: Billing
    result3 = await Runner.run(triage_agent, "I’d like to checkout with my credit card.")
    print(result3.final_output)

# ✅ Run the tests
if __name__ == "__main__":
    asyncio.run(main())

🛍️ Shopping Agent: OK. I found two dresses: Red Evening Dress for $120 and Floral Summer Dress for $80.

📦 Support Agent: We offer a 10% discount on orders above $100.

💳 Billing Agent: OK. I've processed the billing using the default name and email with the provided credit card. A confirmation has been sent to the default email.



# Let's explore now Orchestrator Agent Concept through (as.tool())

In [ ]:
from google.colab import userdata
from openai import AsyncOpenAI
from agents import Agent, Runner, function_tool, set_default_openai_client, set_default_openai_api, set_tracing_disabled

# ✅ Gemini API setup
BASE_URL = 'https://generativelanguage.googleapis.com/v1beta/openai/'
API_KEY = userdata.get('GOOGLE_API_KEY')
MODEL_NAME = "gemini-2.0-flash"

client = AsyncOpenAI(base_url=BASE_URL, api_key=API_KEY)
set_default_openai_client(client=client, use_for_tracing=False)
set_default_openai_api("chat_completions")
set_tracing_disabled(disabled=True)

# ✅ Tools
@function_tool
def list_products(category: str = "all") -> str:
    inventory = {
        "clothing": ["T-Shirt - $20", "Jeans - $50"],
        "accessories": ["Sunglasses - $30", "Hat - $15"],
        "all": ["T-Shirt - $20", "Jeans - $50", "Sunglasses - $30", "Hat - $15"]
    }
    items = inventory.get(category.lower(), inventory["all"])
    return f"OK. I found these products: {', '.join(items)}."

@function_tool
def answer_support_query(issue: str) -> str:
    issue = issue.lower()
    if "return" in issue or "refund" in issue:
        return "We offer a 30-day return policy. Please provide your order number to proceed."
    elif "delivery" in issue or "shipping" in issue:
        return "You can track your order using the link in your confirmation email."
    else:
        return "Please clarify your support issue, and I'll assist you."

# ✅ Agents
shopping_agent = Agent(
    name="Shopping Assistant",
    instructions="""
You assist users in finding products and making purchase decisions.
Use the list_products tool to show available products.
Format your response as: 'Shopping Agent: OK. I found these products: [items].'
    """,
    tools=[list_products],
    model=MODEL_NAME
)

support_agent = Agent(
    name="Support Agent",
    instructions="""
You help users with post-purchase support and returns.
Use the answer_support_query tool to address support issues.
Format your response as: ' Support Agent: [tool response].'
    """,
    tools=[answer_support_query],
    model=MODEL_NAME
)

# Convert agents into tools with required arguments
shopping_tool = shopping_agent.as_tool(
    tool_name="shopping_assistant",
    tool_description="Routes queries about browsing or purchasing products to the Shopping Assistant."
)

support_tool = support_agent.as_tool(
    tool_name="support_assistant",
    tool_description="Routes queries about post-purchase support, returns, or delivery to the Support Agent."
)



# Define an orchestrator agent that delegates tasks
orchestrator_agent = Agent(
    name="Orchestrator Agent",
    instructions="""
You route user queries to the appropriate agent based on their nature:
- If the query involves buying, shopping, products, or categories like clothing or accessories, route to the Shopping Assistant using the shopping_assistant tool.
- If the query involves help, support, returns, refunds, or delivery, route to the Support Agent using the support_assistant tool.
If the query is unclear or doesn't match these categories, respond with: 'Orchestrator Agent: Please clarify your request.'
    """,
    tools=[shopping_tool, support_tool],
    model=MODEL_NAME
)



# ✅ Test the orchestrator agent with sample inputs
def main():
    result1 = Runner.run_sync(orchestrator_agent, "Can you show me some clothing products?")
    print(result1.final_output)

    result2 = Runner.run_sync(orchestrator_agent, "I need help with a recent purchase.")
    print( result2.final_output)

    result3 = Runner.run_sync(orchestrator_agent, "What do you do?")
    print(result3.final_output)

if __name__ == "__main__":
    main()

Shopping Agent: OK. I found these products: OK. I found these products: T-Shirt - $20, Jeans - $50.

Support Agent: What is the issue you are facing with your recent purchase?

Orchestrator Agent: Please clarify your request.

